In [1]:
import pandas as pd
from clean import *

import plotly.express as px

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
data = pd.read_csv('data/profile_data.csv')

In [3]:
data = clean(data)
stats(data)

-----------------
CLEANING DATA
...
FOUND 56783 ENTRIES
REMOVED 0 EMPTY ENTRIES
REMOVED 0 DUPLICATED ENTRIES
-----------------
CALCULATING STATS
...
FOUND 56783 ENTRIES
56335 ENTRIES HAVE AGE DATA (99.2%)
19773 ENTRIES HAVE COLLEGE DATA (34.8%)
14456 ENTRIES HAVE CITY DATA (25.5%)
14540 ENTRIES HAVE JOB DATA (25.6%)
10153 ENTRIES HAVE GENDER DATA (17.9%)
55631 ENTRIES HAVE DISTANCE DATA (98.0%)
42256 ENTRIES HAVE DETAIL DATA (74.4%)
15078 ENTRIES HAVE ANTHEM DATA (26.6%)
...
AVERAGE AGE IN DATA SET: 24
NUMBER OF UNIQUE COLLEGES: 7793
NUMBER OF UNIQUE JOBS: 9530
NUMBER OF UNIQUE CITIES: 2793


In [ ]:
song = data.groupby('anthem').count().name.idxmax()

anthem
("'39 - Remastered 2011", 'Queen')                                                                                                                     1
("'Cause I'm A Man", 'Tame Impala')                                                                                                                    1
("'S Wonderful", 'João Gilberto')                                                                                                                      1
("'פאוץ", 'Noa Kirel')                                                                                                                                 1
("(I Can't Get No) Satisfaction - Mono Version", 'The Rolling Stones')                                                                                 1
("(Joe Gets Kicked Out of School for Using) Drugs With Friends (But Says This Isn't a Problem)", 'Car Seat Headrest')                                  1
("(Sittin' On) The Dock Of The Bay", 'Otis Redding')                       

In [30]:
age_query = int(input("Enter age group:"))
place = data.groupby('city').count().name.idxmax()
print('Most {} year olds live in {}'.format(age_query,place))

Enter age group:35
Most 35 year olds live in Los Angeles


In [18]:
data.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430.0,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...
1,Lauren,21.0,Florida State University,NaN,NaN,NaN,1981.0,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19.0,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18.0,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15.0,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...
